In [2]:
TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)

In [3]:
TWITTER_ACCESS_TOKEN =config('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET =config('TWITTER_ACCESS_TOKEN_SECRET')

# 32 tweets per bert request

In [219]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

def bert_request(timeline):
    tweet_list = [tweet["tweet"]["tweet"] for tweet in timeline]
    data = {"description":tweet_list, "max_seq_length":32}
    headers= {'Content-type':"application/json",
               "cache-control":"no-cache"
              }
    data= json.dumps(data)
    results = requests.post("http://35.232.114.74:5000/", data = data, headers = headers)
    output = [{"tweet":t,"bert_result": r} for t,r in zip(timeline,results.json()["results"])]
    return output
    
def process_tweet(full_tweet):
    tweet = full_tweet.full_text
    #strip username
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    #make api request for toxicity analysis
    
    tweet_info = {"tweet": 
                     {"user_id": full_tweet.user.id,
                      "user_name" : full_tweet.user.name,
                      "tweet": full_tweet.full_text,
                      "tweet_id" : full_tweet.id_str
                     }
                    }
    return tweet_info

def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=32,
                                              tweet_mode='extended',
                                              exlude_rts=False,
                                              exclude_replies=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        output = bert_request(timeline)
        return json.dumps(output)
        
    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
        
def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers['content-type']
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == 'application/json': 
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN' in request_json:
            TWITTER_ACCESS_TOKEN = request_json['TWITTER_ACCESS_TOKEN']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN_SECRET' in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json['TWITTER_ACCESS_TOKEN_SECRET']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")
        
        # Call the function for the POST request. 
        if request.method == 'POST':
            return clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

In [223]:
r = clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
r = json.loads(r)

In [228]:
headers = {"Content-Type":"application/json"}
body = {"TWITTER_ACCESS_TOKEN": config('TWITTER_ACCESS_TOKEN'),
        "TWITTER_ACCESS_TOKEN_SECRET": config('TWITTER_ACCESS_TOKEN_SECRET')
        }

In [232]:
%%time
r = requests.post("https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
                  headers=headers,
                  data=json.dumps(body))

Wall time: 6.78 s


In [233]:
r.json()

[{'tweet': {'tweet': {'user_id': 3765572533,
    'user_name': 'Dr Shawn Baker 🥩',
    'tweet': 'RT @AmericanSoWoke: Is there a connection between the classic dystopian film THX 1138 and the current narrative around food + agriculture?…',
    'tweet_id': '1126679354325340160'}},
  'bert_result': {'identity_hate': 5.924701690673828e-05,
   'insult': 0.00014674663543701172,
   'obscene': 0.00010496377944946289,
   'severe_toxic': 6.365776062011719e-05,
   'threat': 5.1081180572509766e-05,
   'toxic': 0.00024369359016418457}},
 {'tweet': {'tweet': {'user_id': 2895499182,
    'user_name': 'hardmaru',
    'tweet': 'RT @otduet: First attempt at removing cars off the roads with neural nets. Will have to dream harder. https://t.co/DZj63CHo8B',
    'tweet_id': '1126675382436917248'}},
  'bert_result': {'identity_hate': 4.762411117553711e-05,
   'insult': 0.00014919042587280273,
   'obscene': 0.00011119246482849121,
   'severe_toxic': 4.407763481140137e-05,
   'threat': 3.36766242980957e-05,
   '

# single tweet per bert request

In [3]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

def bert_request(tweet_text):
    data = {"description":tweet_text}
    headers= {'Content-type':"application/json",
               "cache-control":"no-cache"
              }
    data= json.dumps(data)
    return requests.post("http://35.192.71.216:5000/", data = data, headers = headers)

def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    #make api request for toxicity analysis
    bert_result = bert_request(tweet).json()
    
    tweet_info = {"tweet": 
                     {"user_id": full_tweet.user.id,
                      "user_name" : full_tweet.user.name,
                      "tweet": full_tweet.full_text,
                      "tweet_id" : full_tweet.id_str,
                     "bert_output": bert_result,
                     "tweet":tweet}
                    }
    return tweet_info
    

def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=20,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
        
def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers['content-type']
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == 'application/json': 
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN' in request_json:
            TWITTER_ACCESS_TOKEN = request_json['TWITTER_ACCESS_TOKEN']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN_SECRET' in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json['TWITTER_ACCESS_TOKEN_SECRET']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")
        
        # Call the function for the POST request. 
        if request.method == 'POST':
            return clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

## TEST - Single

In [53]:
headers = {"Content-Type":"application/json"}
body = {"TWITTER_ACCESS_TOKEN": config('TWITTER_ACCESS_TOKEN'),
        "TWITTER_ACCESS_TOKEN_SECRET": config('TWITTER_ACCESS_TOKEN_SECRET')
        }

In [54]:
%%time
r = requests.post("https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
                  headers=headers,
                  data=json.dumps(body))

Wall time: 30.8 s


In [55]:
r2 = r.json()

In [56]:
r2[0]

{'tweet': {'user_id': 23909137,
  'user_name': 'Larry Brown',
  'tweet': 'Drake, Raptors fans troll Joel Embiid with airplane in Game 5 https://t.co/hjh8KFHR4z',
  'tweet_id': '1125959710362849281',
  'bert_output': {'results': {'identity_hate': 0.00010662349814083427,
    'insult': 0.0002195992274209857,
    'obscene': 0.00018712588644120842,
    'severe_toxic': 0.00010429355461383238,
    'threat': 7.004194776527584e-05,
    'toxic': 0.0005336005124263465}}}}

In [57]:
[r2[i]["tweet"] for i in range(len(r2))]

[{'user_id': 23909137,
  'user_name': 'Larry Brown',
  'tweet': 'Drake, Raptors fans troll Joel Embiid with airplane in Game 5 https://t.co/hjh8KFHR4z',
  'tweet_id': '1125959710362849281',
  'bert_output': {'results': {'identity_hate': 0.00010662349814083427,
    'insult': 0.0002195992274209857,
    'obscene': 0.00018712588644120842,
    'severe_toxic': 0.00010429355461383238,
    'threat': 7.004194776527584e-05,
    'toxic': 0.0005336005124263465}}},
 {'user_id': 8496762,
  'user_name': 'tylercowen',
  'tweet': 'RT @pamela_herd: While Trump should release his tax returns, I am not thrilled that the NYT used de-identified publicly released data to do…',
  'tweet_id': '1125959002880450560',
  'bert_output': {'results': {'identity_hate': 0.00016027066158130765,
    'insult': 0.00021916520199738443,
    'obscene': 0.0001547282445244491,
    'severe_toxic': 0.0001458178012399003,
    'threat': 0.00011751330021070316,
    'toxic': 0.00033231236739084125}}},
 {'user_id': 16396078,
  'user_n

# with Asyncio
did not work

In [1]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

def bert_request(tweet_text):
    data = {"description":tweet_text}
    headers= {'Content-type':"application/json",
               "cache-control":"no-cache"
              }
    data= json.dumps(data)
    return requests.post("http://35.192.71.216:5000/", data = data, headers = headers)

def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    #make api request for toxicity analysis
    bert_result = bert_request(tweet).json()
    
    tweet_info = {"tweet": 
                     {"user_id": full_tweet.user.id,
                      "user_name" : full_tweet.user.name,
                      "tweet": full_tweet.full_text,
                      "tweet_id" : full_tweet.id_str,
                     "bert_output": bert_result,
                     "tweet":tweet}
                    }
    return tweet_info
    

def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=20,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
        
def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers['content-type']
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == 'application/json': 
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN' in request_json:
            TWITTER_ACCESS_TOKEN = request_json['TWITTER_ACCESS_TOKEN']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN_SECRET' in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json['TWITTER_ACCESS_TOKEN_SECRET']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")
        
        # Call the function for the POST request. 
        if request.method == 'POST':
            return clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

In [8]:
def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=200,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    return tweet
    #make api request for toxicity analysis
#     bert_result = bert_request(tweet).json()
    
#     tweet_info = {"tweet": 
#                      {"user_id": full_tweet.user.id,
#                       "user_name" : full_tweet.user.name,
#                       "tweet": full_tweet.full_text,
#                       "tweet_id" : full_tweet.id_str,
#                      "bert_output": bert_result,
#                      "tweet":tweet}
#                     }
#     return tweet_info




In [10]:
%%time
clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)

Wall time: 1.01 s


'["\\u201cWhat do we want?\\u201dAttention!\\u201cWhen do we want it?\\u201dNow!", "lol, oh if he\'s white he\'s definitely not Hispanic then", "", "This is how I start pitches with VCs", "Left: The Onion, 2015Right: The New York Times, 2019", "Some fun late roman honorific portraits from the Stoa of Attalos", "Allzu ernst nimmt sich die Satire-Partei  nicht, wenn sie in den Wahlkampf startet: \\"F\\u00fcr Europa reicht\'s!\\" ist ihr S\\u2026", "Add a caption: What is this lady doing?", "UPDATE: I\'m waiting on confirmation for that thing I got 13K retweets for stating as facth/t", "cc", "An 10 RetterInnen von der #Iuventa soll ein Exempel statuiert werden. Der drohende Prozess auf Sizilien wegen angeblicher Schle\\u2026", "I warned her.", "Despite trashing back pretty hard yesterday with deadlifts (405lbs x 56 reps) was still able to throw down like a beast on the  today- is it all the meat or is the red light\\u2026", "Benutzername \\"dummy\\", Passwort \\"doof\\": Das BfR hat f\\u0

In [67]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
nest_asyncio.apply()

def get_timeline(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=200,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        return home_timeline

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
              

def asycnio_process_tweet(session, full_tweet, tweet_list):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    tweet_list.append(tweet)

async def get_data_asynchronous(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    home_timeline = get_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    tweet_list =[] 
    with ThreadPoolExecutor(max_workers=1) as executor:
        with requests.Session() as session:
            # Set any session parameters here before calling `fetch`

            # Initialize the event loop        
            loop = asyncio.get_event_loop()

            # Use list comprehension to create a list of
            # tasks to complete.
            tasks = [ loop.run_in_executor(
                executor, asycnio_process_tweet,*(session, full_tweet, tweet_list) 
            ) for full_tweet in home_timeline]
            
            # Initializes the tasks to run and awaits their results
            for response in await asyncio.gather(*tasks):
                pass
    return tweet_list
def execute_async_event_loop(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.create_task(get_data_asynchronous(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)

In [70]:
%%time
tweet_list = []
execute_async_event_loop(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

Wall time: 901 ms
